<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/cc-logo-square.png" width="200" alt="cognitiveclass.ai logo" />

<h1>Monte Carlo Control without Exploring Starts Frozen Lake: $\epsilon$ (epsilon)-greedy policies</h1> 

<h2>Table of Contents</h2>
<p>In this lab, you will use Perform Monte Carlo Control without Exploring Starts using $\epsilon$ (epsilon)-greedy policies on the BlackJack environment on open AI gym </p>

<ul>
    <li><a href="#UF">Utility Functions and Imports</a></li>
    <li><a href="#F">Monte Carlo Control without Exploring Starts</a></li>
    <li><a href="#FE">Monte Carlo Prediction Experiments </a></li>
</ul>
<p>Estimated Time Needed: <strong>25 min</strong></p>

<hr>

In [ ]:
#!conda install -c anaconda seaborn
 
#!pip install gym

<h2 id="UF">Utility Functions and Imports </h2> 

In [ ]:
import gym
import matplotlib.pyplot as plt
import math 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import copy
from collections import defaultdict
from deterministicfrozenlake import  FrozenLakeEnv

 <h2 id="F">Exploring Starts Function </h2> 

This function will select the maximum argument of its input,  with the probability of epsilon. Otherwise, it will randomly select the non greedy action.

In [ ]:
class DrawStateGrid():
    def __init__(self,Environment,v=None):
        state_number=4
        self.state_name=[['S','F','F','F'],
        ['F','H','F','H'],
        ['F','F','F','H'],
        ['H','F','F','G']]
        self.numbber=[[0,1,2,3],
        [4,5,6,7],
        [8,9,10,11],
        [12,13,14,15]]

        self.state_action_state=np.zeros((Environment.nS,Environment.nA))
        
        
        for state in range(Environment.nS):
            for action in range(Environment.nA):
                if len(Environment.P[state][action])>1:
                    self.state_action_state[state,action]=Environment.P[state][action][1][1]
                else: 
                       self.state_action_state[state,action]=Environment.P[state][action][0][1]
        
        major_ticks=[i+1 for i in range(state_number)]
        minor_ticks=[i for i in range(state_number)]
        self.fig,self.ax = plt.subplots()


        self.ax.set_xlim([0,state_number])
        self.ax.set_ylim([0,state_number])
        self.ax.grid()
        self.ax.set_xticks(major_ticks)
        self.ax.set_xticks(minor_ticks, minor=True)
        self.ax.set_yticks(major_ticks)
        self.ax.set_yticks(minor_ticks, minor=True)
        n=0 
        for i,row in enumerate(self.state_name):
            for j,val in enumerate(row): 
                self.ax.text(j,4-i-1,"{}:{}".format(val,str(n)),size=25)
           
                n+=1
        
    
    def plot_state(self,start_state,end_state,action):

        #start_state=0
        #end_state=1
        #action=0
        #map states to position on table, top line on square subtract one and its the bottom line
        fig2 = plt.figure()
        y_top=[4,4,4,4,3,3,3,3,2,2,2,2,1,1,1,1]
        
        action_arrow=[(0.5,0),(0,-0.5),(-0.5,0),(0,0.5)]
        
        x_start=np.linspace(start_state%4,start_state%4+1,10)
        self.ax.fill_between(x_start,y_top[start_state]-1,y_top[start_state],color='b')
        
        x_stop=np.linspace(end_state%4,end_state%4+1,10)

        self.ax.fill_between( x_stop,y_top[end_state]-1,y_top[end_state],color='r')
        
     
        self.ax.arrow(x_start[5], y_top[start_state]-0.5, action_arrow[action][0], action_arrow[action][1],head_width=0.05, head_length=0.1, fc='k', ec='k') 
        
    def plot_policy(self,policy):
   
    
        action_arrow=[(-0.5,0),(0,-0.5),(0.5,0),(0,0.5)]
        y_top=[4,4,4,4,3,3,3,3,2,2,2,2,1,1,1,1]    
        for state,action_set in enumerate(policy):
            action=np.argmax(action_set)
            if max(action_set)==1:
                x_start=np.linspace(state%4,state%4+1,10)
                self.ax.arrow(x_start[5], y_top[state]-0.5, action_arrow[action][0], action_arrow[action][1],head_width=0.05, head_length=0.1, fc='k', ec='k')

In [ ]:
def argmax(z,epsilon=0.1):
    """
    This function will select the maximum argument of its input, with the probability of epsilon. Otherwise, it will randomly select the non greedy action
    Returns:
    Args:
    z:numpy array representing the action function 
    epsilon: probability of selecting the non-greedy action 
    policy:policy for blackjack if none a random  action will be selected 
    """
  
    
    argmax_=np.random.choice(np.where(z==z.max())[0])
    
    return argmax_

In [ ]:
def random_action(action,epsilon=0.1,n_actions=4):
    
    number=np.random.rand(1)
    
    if number<1-epsilon:
        return action
    else:
        action=np.random.randint(n_actions)  
        return action 
        
        

The `random_action` function will add noise to the action with a probability of $\epsilon$.

In [ ]:
test=[0,1,2,3,0,0,0,0]
for t in test:
    print("a",t)
    print("a'",random_action(t,epsilon=0.95,n_actions=4))

In [ ]:
def policy_one_hot(policy):
    POLICY=np.zeros((16,4))

    for i,action in  enumerate(policy):

        POLICY[i,int(action)]=1

    return POLICY


The array data has its maximum value at index 0. We set epsilon to 0.75; as a result, it will only select the maximum value  25% of the time. Let's run the experiment 10 times.

We see most of the samples are not the index with the largest values.

This function determines the optimal policy using the  $\epsilon$ (epsilon)-greedy method. The input parameters are the discount factor, the number of episodes, epsilon value  and the open AI gym objects.  The class also Specifies if first-visit  and every-visit method. The output is the policy,value function and action function.

In [ ]:
def monte_carlo_ES( environment,N_episodes=10000,epsilon=0.1, discount_factor=1,first_visit=True,theta=0):
    """
    This function determines the optimal policy using the epsilon greedy method. The input parameters are the discount factor, the number of episodes, epsilon value and the open AI gym objects. The class also Specifies if first-visit and every-visit method. The output is the policy,value function and action function.
    Returns:  
    policy: a dictionary of estimated policy for blackjack 
    V: a dictionary of estimated values for blackjack 
    Q: a dictionary of estimated action function
    DELTA: list of deltas for each episode
    Args:
    environment:AI gym balckjack envorment object 
    N_episodes:number of episodes 
    discount_factor:discount factor
    first_visit: select first-visit MC (Ture) and every-visit MC (False)
    """  
    #a dictionary of estimated values for blackjack 
    V=defaultdict(float)
    #a dictionary of estimated action function for blackjack
    Q=defaultdict(float)
    # number of visits to the action function 
    NumberVisitsValue= defaultdict(float)
    # visits to action function
    NumberVisits= defaultdict(float)
    #dictionary  for policy 
    policy=defaultdict(float) 
    #number  of actions 
    number_actions=environment.action_space.n

    DELTA=[] 
    
    for i in range(N_episodes):
        
        delta=0
        
        #list that stores each state and reward for each episode     
        episode=[]
        # reset the  environment for the next episode and find first state  
        state=environment.reset()   
        #reward for the first state (only for frozen lake) 
        reward=0.0
        #flag for end of episodes  
        done=False
        
         #check if a policy for the state exists  
        if isinstance(policy[state],np.int64):
            #obtain action from policy
            action=policy[state]
            action=random_action(action,epsilon=0.1,n_actions=4)

        else:
            #if no policy for the state exists  select a random  action  
            action=np.random.randint(number_actions)
        
        
        
        #append firt state, reward and action
        episode.append({'state':state , 'reward':reward,'action':action})
        #Past states for signal visit  Monte Carlo 
        state_action=[(state,action)]
        #enumerate for each episode 
        while not(done):

                #take action and find next state, reward and check if the episode is  done (True)
                (state, reward, done, prob) = environment.step(action)

                #check if a policy for the state exists  
                if isinstance(policy[state],np.int64):
                    #obtain action from policy
                    action=int(policy[state])
                    action=random_action(action,epsilon=0.01,n_actions=4)

                else:
                     #if no policy for the state exists  select a random  action  
                    action=np.random.randint(number_actions)
                    
                #add state reward and action to list 
                episode.append({'state':state , 'reward':reward,'action':action})
                #add  states action this is for fist visit only 
                state_action.append((state,action))
         #reverse list as the return is calculated from the last state
        episode.reverse()
        #append the state-action pairs to a list 
        state_action.reverse()


        # determine the return
        G=0
        flag=0
        for t,step in enumerate(episode):
                
                #check flag for first visit
                #G=discount_factor*G+step['reward']
                #if G==1 and flag==0 and t==0:
                    #print(episode)
                    #print("----------------------------------")
                    
                #check flag for first visit
              
                if first_visit:
                    #check if the state has been visited before 
                    
                    if (step['state'],step['action']) not in state_action[t+1:]: 

                        #increment counter for action 
                        NumberVisits[step['state'],step['action']]+=1
                        #increment counter for value function 
                        NumberVisitsValue[step['state']]+=1
                        #if the action function value  does not exist, create an array  to store them 
                        if not(isinstance(Q[step['state']],np.ndarray) ):
                            Q[step['state']]= np.zeros((number_actions))

                        #calculate mean of action function Q Value functions V using the  recursive definition of mean 
                        Q[step['state']][step['action']]=Q[step['state']][step['action']]+(NumberVisits[step['state'],step['action']]**-1)*(G-Q[step['state']][step['action']])
                        
                        v=V[step['state']]
                        
                        V[step['state']]=V[step['state']]+(NumberVisitsValue[step['state']]**-1)*(G-V[step['state']])
                        ##update the policy to select the action fuciton argment with the largest value or randomly select a different action 
                        policy[step['state']]=np.random.choice(np.where(Q[step['state']]==Q[step['state']].max())[0])
                        
                         
                        #find max difference between all value functions per  iteration 
                        delta=max(delta,abs(v-V[step['state']]))
                        
                else:
                         #increment counter for action 
                        NumberVisits[step['state'],step['action']]+=1
                        #increment counter for value function 
                        NumberVisitsValue[step['state']]+=1
                        #if the action function value  does not exist, create an array  to store them 
                        if not(isinstance(Q[step['state']],np.ndarray) ):
                            Q[step['state']]= np.zeros((number_actions))

                        #calculate mean of action function Q Value functions V using the  recursive definition of mean 
                        Q[step['state']][step['action']]=Q[step['state']][step['action']]+(NumberVisits[step['state'],step['action']]**-1)*(G-Q[step['state']][step['action']])
                        #old value of value function
                        v=V[step['state']]
                        V[step['state']]=V[step['state']]+(NumberVisitsValue[step['state']]**-1)*(G-V[step['state']])
                        ##update the policy to select the action fuciton argment with the largest value randomly select a different action 
                        policy[step['state']]=np.random.choice(np.where(Q[step['state']]==Q[step['state']].max())[0])
                        
                        #find max difference between all value functions per  iteration 
                        delta=max(delta,abs(v-V[step['state']]))
               
            
                G=discount_factor*G+step['reward']
        
        #Delta for each episode  
        DELTA.append(delta)
        if delta<theta:
            break
         
                    

    
    return policy, V, Q,DELTA

 <h2 id="FE">Monte Carlo Prediction Experiments  </h2> 

Let’s  run different iterations of and see how different  parameters such as number of episodes  and discount factor effects performance.

We create an openAI gym blackjack enviroment:

In [ ]:
gym.envs.register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.78, # optimum = .8196
)
environment=gym.make('FrozenLakeNotSlippery-v0')

Let's see what happens if we use a random policy:

Lets perform  $\epsilon$ -greedy method  with 1000 episodes and  $\epsilon$=0.1 :

<p>LEFT = 0</p>
<p>DOWN = 1</p>
<p>RIGHT = 2</p>
<p>UP = 3</p>

In [ ]:
policy, V, Q,DELTA= monte_carlo_ES( environment,N_episodes=1000,epsilon=0.01, discount_factor=1,first_visit=True,theta=0)  
policy

In [ ]:
plt.plot(DELTA)

In [ ]:
environment.reset()

We will create an `actions` array for purposes of plotting the policy. 

In [ ]:
actions=np.zeros(16)
for i,action in zip(policy.keys(),policy.values()):
    actions[i]=action
    
policy_one_hot(actions)

#print(Q)
policy

<p>LEFT = 0</p>
<p>DOWN = 1</p>
<p>RIGHT = 2</p>
<p>UP = 3</p>

The policy is shown on the grid as arrows.

In [ ]:
test=DrawStateGrid(environment.env,np.zeros(15))
test.plot_policy(policy_one_hot(actions))
actions

The Action function $q_{\pi}(s, a)$ reports the value for action $a$ at state $s$ using the policy $\pi$ 

In [ ]:
Q

The policy is displayed along with the one-hot encoded version.

In [ ]:
print(actions)
print(policy_one_hot(actions))

We show the optimal policy for blackjack found by Monte Carlo exploring starts; blue represents the white represents a stick:

It's difficult to determine a policy; this is because we did not specify the appropriate number of episodes. We can plot out the action function:

<b>About the Authors:</b> 

<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.

<b>References</b>

<ol type="1">
    <li>Sutton, Richard S., and Andrew G. Barto. "Reinforcement learning: An introduction[</li>
    <li><a href="https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py">blackjack openai Gym</a></li>
   

</ol>

Copyright &copy; 2020 <a href="cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu">cognitiveclass.ai</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.